---
title: "Using Manopt.jl from within JuMP"
author: Ronny Bergmann
---


In this tutorial we aim to illustrate how to use manifolds and the algorithms from [`Manopt.jl`](https://manoptjl.org) within the [JuMP](https://jump.dev) framework by implementing the [get started](getstarted.md) tutorial again using JuMP combining it with for example their [get started](https://jump.dev/JuMP.jl/stable/tutorials/getting_started/getting_started_with_JuMP/#Getting-started-with-JuMP) tutorial.


In [ ]:
#| echo: false
#| code-fold: true
#| output: false
using Pkg;
cd(@__DIR__)
Pkg.activate("."); # for reproducibility use the local tutorial environment.

In [ ]:
using JuMP, Manopt, Manifolds

First we generate the same data for the [Riemannian center of mass]() as before


In [ ]:
n = 100
σ = π / 8
M = Sphere(2)
p = 1 / sqrt(2) * [1.0, 0.0, 1.0]
data = [exp(M, p,  σ * rand(M; vector_at=p)) for i in 1:n];

and we start by stating that our [JuMP model](https://jump.dev/JuMP.jl/stable/manual/models/) is


In [ ]:
model = Model(Manopt.JuMP_Optimizer)

Next we add a [JuMP variable](https://jump.dev/JuMP.jl/stable/manual/variables/),
where we specify the manifold using the `in` keyword, `start=` cares for the initialisation


In [ ]:
@variable(model, p[i=1:3] in M, start=p[i])

For now this is restricted to array-type-representations of points and tangent vectors.

## Variant I: Specify the

One can specify a cost in the embedding and get an approximate gradient in the embedding that is “converted” into a Riemannian one, see [AD](AutomaticDifferentiation.md).


In [ ]:
f(p) = sum(1 / (2 * n) * norm.(Ref(p) - data) .^ 2)
grad_f(p) = sum(1 / n * grad_distance.(Ref(M), data, Ref(p)));
# TODO: how to set the objective to an implemented function ?
# TODO 100% unsure how to set a gradient since that seems to be nowhere documented ?!
@objective(model, f)

We set the solver to use to be [gradient descent](gradient_descent.md)


In [ ]:
set_attribute("descent_state_type", GradientDescentState)

Then we can call the solver


In [ ]:
optimize!(model)

and look at the solution


In [ ]:
solution_summary(model)

## Vriant II: Specify the objective directly

The [objective](https://jump.dev/JuMP.jl/stable/manual/objective/) is set for example using our [Objectives](objective.md).


In [ ]:
model2 = Model(Manopt.JuMP_Optimizer)
@variable(model2, p[i=1:3] in M, start = p[i])
f(M, p) = sum(1 / (2 * n) * distance.(Ref(M), Ref(p), data) .^ 2)
grad_f(M, p) = sum(1 / n * grad_distance.(Ref(M), data, Ref(p)));
# ToDo: Does not yet work, but I thought it should by now?
@objective(model2, Min, Manopt.ManifoldGradientObjective(f, grad_f))
set_attribute("descent_state_type", GradientDescentState)

And we issue again the optimization


In [ ]:
optimize!(model2)

and look at the solution


In [ ]:
solution_summary(model2)